In [540]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [541]:
import random

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold, GridSearchCV

from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings
warnings.filterwarnings('ignore')

matplotlib.rcParams.update({'font.size': 14})

In [542]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

In [543]:
TRAIN_DATASET_PATH = '../input/real-estate-price-prediction-moscow/train.csv'
TEST_DATASET_PATH = '../input/real-estate-price-prediction-moscow/test.csv'

Просматриваем датасеты для обучения и теста

In [544]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
train_df.head()

In [545]:
train_df.dtypes

In [546]:
test_df = pd.read_csv(TEST_DATASET_PATH)
test_df.head()

Убеждаемся, что количество признаков в датасетах совпадает (без цены в train) 

In [547]:
train_df.shape[1] - 1 == test_df.shape[1]

Меняем вещественные признаки, величина значения которых не влияет на конечный признак, на категориальные:

In [548]:
train_df['Id'] = train_df['Id'].astype(str)
train_df['DistrictId'] = train_df['DistrictId'].astype(str)

Изучаем целевую переменную:

In [549]:
plt.figure(figsize = (8,4))
train_df['Price'].hist(bins=40)
plt.xlabel('Price')
plt.ylabel('Count')

plt.show()

### **Исследуем признаки:**

In [550]:
train_df.describe()

In [551]:
train_df.count()

In [552]:
train_df.dtypes

 ### **Обрабатываем вещественные переменные.**

**Исследуем признак Rooms**

Обрабатываем неадекватные значения кол-ва комнат:

In [553]:
train_df['Rooms'].value_counts()

In [554]:
train_df['Rooms_outlier'] = 0
train_df.loc[((train_df['Rooms'] == 0) | (train_df['Rooms'] >= 6)), 'Rooms_outlier'] = 1

Нулевые значения приравниваем к одной комнате.
Слишком большие значения комнат - к медиане.

In [555]:
train_df.loc[train_df['Rooms'] == 0, 'Rooms'] = 1
train_df.loc[train_df['Rooms'] >= 6, 'Rooms'] = train_df['Rooms'].median()

In [556]:
train_df['Rooms'].value_counts()

**Исследуем признак Square:**

In [557]:
plt.figure(figsize = (8,4))
train_df['Square'].hist(bins=40)
plt.xlabel('Square')
plt.ylabel('Count')

plt.show()

"Хвост" с общей площадью больше 140 небольшой, можно было бы его обрубить, но в Московской области площадь самых больших квартир превышает 800. Значит наши значения вполне реальны, оставляем все значения.

In [558]:
train_df['Square'].loc[train_df['Square'] < 10].value_counts().sum()

Минимальная жилплощадь - 10 кв м. Приравняем все площади меньше 10 к минимальному значению:

In [559]:
train_df.loc[train_df['Square'] < 10, 'Square'] = 10

**Исследуем признак KitchenSquare:**

In [560]:
train_df['KitchenSquare'].value_counts()

Нижнюю границу обрезаем по адекватному значению (3), верхнюю - по квантилю 0.975:

In [561]:
condition_K = (train_df['KitchenSquare'].isna()) | (train_df['KitchenSquare'] > train_df['KitchenSquare'].quantile(.975))
        
train_df.loc[condition_K, 'KitchenSquare'] = train_df['KitchenSquare'].median()

train_df.loc[train_df['KitchenSquare'] < 3, 'KitchenSquare'] = 3

**Исследуем признак LifeSquare:**

In [562]:
train_df['LifeSquare'].sort_values(ascending = True)

Самые большие квартиры в Московской области имеют площадь 800 с хвостиком.
Меньше, чем на 3 кв м кровать поставить сложно.
Неадекватные значения и пропуски заполняем разницей общей площади и кухни+санузла

In [563]:
train_df['LifeSquare_outlier'] = 0
train_df.loc[( (train_df['LifeSquare'] < 3) | (train_df['LifeSquare'] >= 900) | (train_df['LifeSquare'].isna())), 'LifeSquare_outlier'] = 1

In [564]:
condition_L = ((train_df['LifeSquare'].isna()) | (train_df['LifeSquare'] > 900) | (train_df['LifeSquare'] < 3)) \
                  & (~train_df['Square'].isna()) & (~train_df['KitchenSquare'].isna())
        
train_df.loc[condition_L, 'LifeSquare'] = train_df.loc[condition_L, 'Square'] \
                                            - train_df.loc[condition_L, 'KitchenSquare'] - 3


In [565]:
train_df.sort_values(by=['LifeSquare'])

**Исследуем признак HouseFloor, Floor:**

In [566]:
train_df['HouseFloor'].sort_values(ascending = False)

In [567]:
train_df['Floor'].sort_values(ascending = False)

Значения этажей адекватные.

Обрабатываем этажность: верхнюю границу обрезаем по 95 (самое высокое здание в Московской области), нулевые значения приравниваем к медиане.

Обрабатываем этажи, которые больше этажности: даём рандомное значение.

In [568]:
train_df['HouseFloor_outlier'] = 0
train_df.loc[train_df['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
train_df.loc[train_df['Floor'] > train_df['HouseFloor'], 'HouseFloor_outlier'] = 1

In [569]:
train_df.loc[train_df['HouseFloor'] == 0, 'HouseFloor'] = train_df['HouseFloor'].median()
train_df.loc[train_df['HouseFloor'] > 95, 'HouseFloor'] = 95

In [570]:
floor_outliers = train_df.loc[train_df['Floor'] > train_df['HouseFloor']].index
train_df.loc[floor_outliers, 'Floor'] = train_df.loc[floor_outliers, 'HouseFloor']\
                                                .apply(lambda x: random.randint(1, x))

**Исследуем признак HouseYear:**

In [571]:
train_df['HouseYear'].sort_values(ascending=False)

Слишком большие значения заменяем нынешним годом:

In [572]:
current_time = datetime.now().year
train_df.loc[train_df['HouseYear'] > current_time, 'HouseYear'] = current_time

**Исследуем признаки Social, Healthcare, Social, Shops:**

На это времени не хватило :(
Но были идеи проанализировать признаки с помощью матрицы корреляций (видна обратная зависимость некоторых признаков от размера района и прямая между друг другом)

In [573]:
plt.figure(figsize = (8,4))
train_df['Social_3'].hist(bins=40)
plt.xlabel('Square')
plt.ylabel('Count')
plt.title('Social_3')
plt.show()

In [574]:
train_df['Social_3'].loc[train_df['Social_3'] > 100].value_counts()

In [575]:
# import seaborn as sns
# plt.figure(figsize = (20,14))
# hm = sns.heatmap(X_train[feature_names + new_feature_names].corr(),
#                  cbar=True,
#                  annot=True,
#                 fmt='.1g')

## **Обработаем пропуски:**

In [576]:
train_df.isna().sum()

In [577]:
train_df.loc[train_df['Healthcare_1'].isna(), 'Healthcare_1'] = train_df['Healthcare_1'].median()

## **Оформление препроцессинга в класс**

In [578]:
class DataPreprocessing:
    """Подготовка исходных данных"""

    def __init__(self):
        """Параметры класса"""
        self.medians = None
        self.kitchen_square_quantile = None
        
    def fit(self, X):
        """Сохранение статистик"""       
        # Расчет медиан
        self.medians = X.median()
        self.kitchen_square_quantile = X['KitchenSquare'].quantile(.975)
    
    def transform(self, X):
        """Трансформация данных"""

        # Rooms
        X['Rooms_outlier'] = 0
        X.loc[(X['Rooms'] == 0) | (X['Rooms'] >= 6), 'Rooms_outlier'] = 1
        
        X.loc[X['Rooms'] == 0, 'Rooms'] = 1
        X.loc[X['Rooms'] >= 6, 'Rooms'] = self.medians['Rooms']
        
        #Square
        X.loc[X['Square'] < 10, 'Square'] = 10
        
        # KitchenSquare
        condition_K = (X['KitchenSquare'].isna()) \
                    | (X['KitchenSquare'] > self.kitchen_square_quantile)
        
        X.loc[condition_K, 'KitchenSquare'] = self.medians['KitchenSquare']

        X.loc[X['KitchenSquare'] < 3, 'KitchenSquare'] = 3
        
        # LifeSquare
        X['LifeSquare_outlier'] = 0
        X.loc[(X['LifeSquare'] < 3) \
               | (X['LifeSquare'] >= 900) \
               | (X['LifeSquare'].isna()), 'LifeSquare_outlier'] = 1
        
        condition_L = ((X['LifeSquare'].isna()) | (X['LifeSquare'] > 900)) & \
                      (~X['Square'].isna()) & \
                      (~X['KitchenSquare'].isna())
        
        X.loc[condition_L, 'LifeSquare'] = X.loc[condition_L, 'Square'] - X.loc[condition_L, 'KitchenSquare'] - 3
        
        # HouseFloor, Floor
        X['HouseFloor_outlier'] = 0
        X.loc[(X['HouseFloor'] == 0) | (X['HouseFloor'] == 95), 'HouseFloor_outlier'] = 1
        X.loc[X['Floor'] > X['HouseFloor'], 'HouseFloor_outlier'] = 1
        
        X.loc[X['HouseFloor'] == 0, 'HouseFloor'] = self.medians['HouseFloor']
        X.loc[X['HouseFloor'] > 95, 'HouseFloor'] = 95
        
        floor_outliers = X.loc[X['Floor'] > X['HouseFloor']].index
        X.loc[floor_outliers, 'Floor'] = X.loc[floor_outliers, 'HouseFloor']\
                                            .apply(lambda x: random.randint(1, x))
        
        # HouseYear
        current_time = datetime.now().year
        
        X['HouseYear_outlier'] = 0
        X.loc[X['HouseYear'] > current_time, 'HouseYear_outlier'] = 1
        
        X.loc[X['HouseYear'] > current_time, 'HouseYear'] = current_time
                
        
        
        
        X.fillna(self.medians, inplace=True)
        
        return X

## **Новые признаки**

Бинарные признаки заменяем численными значениями:

In [579]:
binary_to_numbers = {'A': 0, 'B': 1}

train_df['Ecology_2'] = train_df['Ecology_2'].replace(binary_to_numbers)
train_df['Ecology_3'] = train_df['Ecology_3'].replace(binary_to_numbers)
train_df['Shops_2'] = train_df['Shops_2'].replace(binary_to_numbers)

Номер района ни о чём не говорит, но по количеству упоминаний номера района можно судить о его размере. Введём признак Размер района:

In [580]:
district_size = train_df['DistrictId'].value_counts().reset_index()\
                    .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})

district_size.head()

In [581]:
train_df = train_df.merge(district_size, on='DistrictId', how='left')
train_df.head()

In [582]:
train_df['DistrictSize'].max()

Разобьём районы по категориям в соответствии с их размерами:

In [583]:
def dist_size_to_cat(X):
    bins = [0, 200, 400, X['DistrictSize'].max()]
    X['DistrictSize_cat'] = pd.cut(X['DistrictSize'], bins=bins, labels=False)
    
    X['DistrictSize_cat'].fillna(-1, inplace=True)
    return X

In [584]:
train_df = dist_size_to_cat(train_df)
train_df.head()

**MedPriceByYear**



Цена по идее зависит от того, насколько дом старый,и от этажности (в разные года строили дома разной этажности; например, в начале ХХ века не очень популярны были высотки, вряд ли их умели качественно строить, поэтому квартира в старом и высоком доме скорее всего будет иметь низкую стоимость).
Разобьем на категории возраст дома и этажность и совместим их в один признак

In [585]:
def year_to_cat(X):
    bins = [1000, 1941, 1945, 1980, 2000, 2010, X['HouseYear'].max()]
    X['year_cat'] = pd.cut(X['HouseYear'], bins=bins, labels=False)
    
    X['year_cat'].fillna(-1, inplace=True)
    return X

def housefloor_to_cat(X):
    bins = [0, 3, 10, 25, X['HouseFloor'].max()]
    X['housefloor_cat'] = pd.cut(X['Floor'], bins=bins, labels=False)
    
    X['housefloor_cat'].fillna(-1, inplace=True)
    return X

In [586]:
train_df = year_to_cat(train_df)
train_df = housefloor_to_cat(train_df)
train_df.head()

In [587]:
med_price_by_housefloor_year = train_df.groupby(['year_cat', 'housefloor_cat'], as_index=False).agg({'Price':'median'}).\
                                            rename(columns={'Price':'MedPriceByHouseFloorYear'})
med_price_by_housefloor_year.head()

In [588]:
train_df = train_df.merge(med_price_by_housefloor_year, on=['year_cat', 'housefloor_cat'], how='left')
train_df.head()

**MedPriceByDistSquare**

Стоимость квартиры также зависит от площади и популярности района.
Разобьём на категории размер района (т.е. его популярность) и площадь квартир и объединим их в один признак.

In [589]:
def square_to_cat(X):
    bins = [0, 30, 60, 90, 150, X['Square'].max()]
    X['Square_cat'] = pd.cut(X['Square'], bins=bins, labels=False)
    
    X['Square_cat'].fillna(-1, inplace=True)
    return X

In [590]:
train_df = square_to_cat(train_df)

In [591]:
med_price_by_distsize_square = train_df.groupby(['Square_cat', 'DistrictSize_cat'], as_index=False).agg({'Price':'median'})\
                            .rename(columns={'Price':'MedPriceByDistSquare'})

med_price_by_distsize_square.head()

In [592]:
train_df = train_df.merge(med_price_by_distsize_square, on=['Square_cat', 'DistrictSize_cat'], how='left')
train_df.head()

### **Оформление генерации новых фич в класс**

In [593]:
class FeatureGenetator():
    """Генерация новых фич"""
    
    def __init__(self):
        self.DistrictId_counts = None
        self.binary_to_numbers = None
        self.med_price_by_distsize_square = None
        self.med_price_by_housefloor_year = None
        self.district_size_max = None
        self.house_year_max = None
        self.house_floor_max = None
        self.square_max = None
        self.district_size = None
        
    def year_to_cat(self, X):
        bins = [1000, 1941, 1945, 1980, 2000, 2010, self.house_year_max]
        X['year_cat'] = pd.cut(X['HouseYear'], bins=bins, labels=False)
    
        X['year_cat'].fillna(-1, inplace=True)
        return X
    
    def fit(self, X, y=None):
        
        X = X.copy()
        
        # Binary features
        self.binary_to_numbers = {'A': 0, 'B': 1}
        
        # DistrictID
        self.district_size = X['DistrictId'].value_counts().reset_index() \
                               .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})
        X = X.merge(self.district_size, on='DistrictId', how='left')
                
        # Target encoding
        ## HouseFloor, Year
        df = X.copy()
        
        if y is not None:
            self.house_year_max = df['HouseYear'].max()
            self.house_floor_max = df['HouseFloor'].max()
            df['Price'] = y.values
            df = self.year_to_cat(df)
            df = self.housefloor_to_cat(df)
            self.med_price_by_housefloor_year = df.groupby(['year_cat', 'housefloor_cat'], as_index=False).agg({'Price':'median'}).\
                                            rename(columns={'Price':'MedPriceByHouseFloorYear'})
                                 
            self.med_price_by_housefloor_year_median = self.med_price_by_housefloor_year['MedPriceByHouseFloorYear'].median()
            
        ## DistrictSize, Square
        if y is not None:
            self.square_max = df['Square'].max()
            self.district_size_max = df['DistrictSize'].max()
            df['Price'] = y.values
            df = self.square_to_cat(df)
            df = self.dist_size_to_cat(df)
            self.med_price_by_distsize_square = df.groupby(['Square_cat', 'DistrictSize_cat'], as_index=False).agg({'Price':'median'}).\
                                            rename(columns={'Price':'MedPriceByDistSizeSquare'})
            self.med_price_by_distsize_square_median = self.med_price_by_distsize_square['MedPriceByDistSizeSquare'].median()
        

        
    def transform(self, X):
        
        # Binary features
        X['Ecology_2'] = X['Ecology_2'].map(self.binary_to_numbers)  # self.binary_to_numbers = {'A': 0, 'B': 1}
        X['Ecology_3'] = X['Ecology_3'].map(self.binary_to_numbers)
        X['Shops_2'] = X['Shops_2'].map(self.binary_to_numbers)
        
        # DistrictId
        if 'DistrictSize' not in X:
            X = X.merge(self.district_size, on='DistrictId', how='left')
        
        X['new_district'] = 0
        X.loc[X['DistrictSize'].isna(), 'new_district'] = 1
        
        X['DistrictSize'].fillna(5, inplace=True)
               
        
        # More categorical features
        X = self.housefloor_to_cat(X)  # + столбец floor_cat
        X = self.year_to_cat(X)   # + столбец year_cat
        X = self.square_to_cat(X)
        X = self.dist_size_to_cat(X)
        
        # Target encoding
        if self.med_price_by_housefloor_year is not None:
            X = X.merge(self.med_price_by_housefloor_year, on=['year_cat', 'housefloor_cat'], how='left')
            X['MedPriceByHouseFloorYear'].fillna(self.med_price_by_housefloor_year_median, inplace=True)
            
        if self.med_price_by_distsize_square is not None:
            X = X.merge(self.med_price_by_distsize_square, on=['Square_cat', 'DistrictSize_cat'], how='left')
            X['MedPriceByDistSizeSquare'].fillna(self.med_price_by_distsize_square_median, inplace=True)
        
        return X
    
    

    def housefloor_to_cat(self, X):
        bins = [0, 3, 10, 25, self.house_floor_max]
        X['housefloor_cat'] = pd.cut(X['Floor'], bins=bins, labels=False)
    
        X['housefloor_cat'].fillna(-1, inplace=True)
        return X
    def square_to_cat(self, X):
        bins = [0, 30, 60, 90, 150, self.square_max]
        X['Square_cat'] = pd.cut(X['Square'], bins=bins, labels=False)
        X['Square_cat'].fillna(-1, inplace=True)
        return X

    def dist_size_to_cat(self, X):
        bins = [0, 200, 400, self.district_size_max]
        X['DistrictSize_cat'] = pd.cut(X['DistrictSize'], bins=bins, labels=False)
        X['DistrictSize_cat'].fillna(-1, inplace=True)
        return X



Выбираем признаки, по которым будем строить модель:

In [594]:
train_df.columns.tolist()

In [595]:
feature_names = ['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear',
                 'Ecology_1', 'Ecology_2', 'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1',
                 'Helthcare_2', 'Shops_1', 'Shops_2']

new_feature_names = ['Rooms_outlier', 'HouseFloor_outlier', 'DistrictSize', 'DistrictSize_cat',
                     'year_cat', 'housefloor_cat',  'MedPriceByHouseFloorYear', 'Square_cat', 'MedPriceByDistSizeSquare']

target_name = 'Price'

Разбиваем исходные данные на train и valid:

In [596]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
test_df = pd.read_csv(TEST_DATASET_PATH)

X = train_df.drop(columns=target_name)
y = train_df[target_name]

In [597]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=26)

In [598]:
preprocessor = DataPreprocessing()
preprocessor.fit(X_train)

X_train = preprocessor.transform(X_train)
X_valid = preprocessor.transform(X_valid)
test_df = preprocessor.transform(test_df)

X_train.shape, X_valid.shape, test_df.shape

In [599]:
features_gen = FeatureGenetator()
features_gen.fit(X_train, y_train)

X_train = features_gen.transform(X_train)
X_valid = features_gen.transform(X_valid)
test_df = features_gen.transform(test_df)

X_train.shape, X_valid.shape, test_df.shape

In [600]:
X_train = X_train[feature_names + new_feature_names]
X_valid = X_valid[feature_names + new_feature_names]
test_df = test_df[feature_names + new_feature_names]

In [601]:
X_train.isna().sum().sum(), X_valid.isna().sum().sum(), test_df.isna().sum().sum()

In [602]:
X_train.shape, X_valid.shape, test_df.shape

Проверяем разные модели. Оставляем RandomForestRegressor как наименее переобучаемый.
С помощью GridSearch подбираем оптимальные параметры.

In [603]:
parameters = [{'n_estimators':range(70,90,1),
             'max_features': [9],
             'max_depth': [11]}]

In [604]:
rfr = RandomForestRegressor(n_estimators=81,
                                   max_depth=8,
                                   max_features=8,
                                   random_state=26)
rfr.fit(X_train, y_train)
y_train_preds = rfr.predict(X_train)
y_test_preds = rfr.predict(X_valid)

evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds)

In [605]:
cv_score = cross_val_score(rfr, X_train, y_train, scoring='r2', cv=KFold(n_splits=3, shuffle=True, random_state=26))
cv_score

In [606]:
cv_score.mean()

In [607]:
feature_importances = pd.DataFrame(zip(X_train.columns, rfr.feature_importances_), 
                                   columns=['feature_name', 'importance'])

feature_importances.sort_values(by='importance', ascending=False)

### **Тестовый датасет**

In [608]:
test_df.shape

In [609]:
test_df

In [610]:
predictions = rfr.predict(test_df)
predictions

In [611]:
submit = pd.read_csv('/kaggle/input/real-estate-price-prediction-moscow/sample_submission.csv')
submit.head()

In [612]:
submit['Price'] = predictions
submit.head()

In [613]:
submit.to_csv('rfr_submit.csv', index=False)